## 01. 라이브러리 로드

In [ ]:
import pandas as pd
import numpy as np

# SMOTE - BorderlineSMOTE만 사용
from imblearn.over_sampling import BorderlineSMOTE
from collections import Counter

# 머신러닝 모델 및 평가
from sklearn.preprocessing import RobustScaler
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from sklearn.feature_selection import f_classif

# 경고 메시지 무시
import warnings
warnings.filterwarnings('ignore')

## 02. 데이터 로드

In [31]:
df = pd.read_excel("data/label_dataset.xlsx",dtype={'거래소코드':str})

## 03. 독립변수 / 종속변수 정의

In [32]:
# 독립변수(피처)와 종속변수(타겟) 컬럼 이름 정의

feature_columns = [col for col in df.columns if col not in ['회사명', '거래소코드', '회계년도', '이자보상배율(이자비용)',
                                                            '영업활동으로 인한 현금흐름(*)(천원)', 'target_class']]

print("사용될 피처 개수:", len(feature_columns))

사용될 피처 개수: 27


## 04. train test 분할
- train : 2013년 ~ 2019년
- test : 2020년 ~ 2022년

In [33]:
split_year = 2019

train = df[df['회계년도'] <= split_year]
test = df[df['회계년도'] > split_year]

X_train = train[feature_columns]
y_train = train['target_class']

X_test = test[feature_columns]
y_test = test['target_class']

## 05. 데이터 스케일링
- RobustScaler 사용 - 이상치에 영향을 줄이기 위해 사용

In [34]:
scaler = RobustScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## 06. 불균형 데이터 처리

#### 원본 라벨 분포 확인

In [35]:
print("원본데이터 : ", Counter(y_train))

원본데이터 :  Counter({0: 9025, 2: 1445, 3: 1174, 1: 479})


#### SMOTE 적용 후 라벨 분포 확인
- borderline-2 : 소수 클래스와 다수 클래스 이웃 모두와 합성 데이터를 생성하는 오버샘플링 기법이기에 사용

In [36]:
min_class_samples = min(Counter(y_train).values())
safe_k = min(5, max(1, min_class_samples - 1))

borderline_smote2 = BorderlineSMOTE(
            random_state=42, 
            kind='borderline-2',
            k_neighbors=safe_k,
            m_neighbors=max(5, safe_k)
        )

X_train_resampled, y_train_resampled = borderline_smote2.fit_resample(X_train_scaled, y_train)
print("SMOTE 적용 이후 데이터:", Counter(y_train_resampled))

SMOTE 적용 이후 데이터: Counter({0: 9025, 2: 9025, 3: 9025, 1: 9025})


## 07. 분산분석(ANOVA) - f_classif 함수 이용
- 분산분석을 통해 y와의 관계를 검정
- p-value < 0.05 인 변수는 유의함 -> 분석에 사용 가능함

In [37]:
# ANOVA (f_classif) 수행
F_values, p_values = f_classif(X_train_resampled, y_train_resampled)

anova_results = pd.DataFrame({
    'Feature': feature_columns,
    'F_value': F_values,
    'p_value': p_values
})

# 유의수준 0.05 기준 유의미한 피처 필터링
significant_features = anova_results[anova_results['p_value'] < 0.05]
non_significant_features = anova_results[anova_results['p_value'] >= 0.05]

print("유의미한 피처 (p-value < 0.05):")
print(significant_features[['Feature', 'p_value']])

print("\n무의미한 피처 (p-value >= 0.05):")
print(non_significant_features[['Feature', 'p_value']])

유의미한 피처 (p-value < 0.05):
         Feature        p_value
0   매출원가 대 매출액비율   0.000000e+00
1        총자본순이익률   0.000000e+00
2         차입금의존도   0.000000e+00
3           당좌비율   1.860205e-39
4       자기자본구성비율   0.000000e+00
5        자기자본회전률   1.210867e-48
6         총자본회전률   0.000000e+00
7      정상영업이익증가율  1.021562e-217
8         순이익증가율  2.036491e-161
9         매출액증가율   1.110903e-53
10            x2   0.000000e+00
11            x3   0.000000e+00
12            x4   2.134513e-93
13           pcr   3.151210e-02
14           psr   1.029026e-07
15       ln(매출액)   0.000000e+00
16      ln(시가총액)   0.000000e+00
17      ln(기업업력)   6.751667e-84
18     잉여현금흐름 비율   0.000000e+00
19         영업이익률   6.800569e-03
20    적자_누적_flag   0.000000e+00
21  기업유동성위험_flag  8.909897e-306
22      재무레버리지효과   9.992293e-41
23       유동자산_비율   0.000000e+00
24   자산_EBITDA비율   1.971394e-35
25     유동부채_자본비율   3.933899e-15
26        유동부채비율   1.218003e-04

무의미한 피처 (p-value >= 0.05):
Empty DataFrame
Columns: [Feature, p_value]
Index:

## 08. 변수중요도 산출
- 아래 트리모형들 중 가장 성능이 좋은 모형의 변수중요도 채택
- 트리기반 앙상블 모델
  - Random Forest
  - XGBoost
  - LightGBM
  - CatBoost
  - Extra Trees

#### 모델 별 확인 결과 CatBoost 성능이 좋아 채택

In [38]:
model = CatBoostClassifier(
            iterations=400,             # 반복 횟수 증가
            depth=8,                    # 깊이 증가
            learning_rate=0.02,         # 학습률 감소
            l2_leaf_reg=2,              # L2 정규화 감소
            border_count=128,           # 경계 개수 증가
            bagging_temperature=0.3,    # 배깅 온도 감소
            random_strength=0.3,        # 무작위 강도 감소
            od_type='IncToDec',
            od_wait=100,                # 조기 중단 대기 증가
            rsm=0.85,                   # 무작위 서브스페이스 방법
            grow_policy='SymmetricTree', # 성장 정책
            bootstrap_type='Bayesian',   # 부트스트랩 타입
            sampling_frequency='PerTreeLevel', # 샘플링 빈도
            leaf_estimation_method='Newton', # 리프 추정 방법
            score_function='Cosine',     # 점수 함수
            objective='MultiClass',
            auto_class_weights='Balanced', # 자동 클래스 가중치
            random_state=42,
            thread_count=-1,
            verbose=False
        )

model.fit(X_train_resampled, y_train_resampled)
importances = model.feature_importances_

# 예측값 생성
y_pred = model.predict(X_test_scaled)
y_proba = model.predict_proba(X_test_scaled)

print("정확도(Accuracy):", accuracy_score(y_test, y_pred))
print("F1 score:", f1_score(y_test, y_pred, average='macro', zero_division=0))
print("AUC:", roc_auc_score(y_test, y_proba, multi_class='ovr'))

print(f"\n=== 전체 {len(importances)}개 피처의 변수 중요도 ===")
print("(중요도 순으로 정렬)")

feature_importance_pairs = list(zip(feature_columns, importances))
feature_importance_pairs.sort(key=lambda x: x[1], reverse=True)

for rank, (name, importance) in enumerate(feature_importance_pairs, 1):
    print(f"{rank:2d}. {name}: {importance:.4f}")

selected_features = [name for name, importance in feature_importance_pairs[:10]]

print(f"\n=== 변수 중요도 (상위 10개) ===")
for rank, (name, importance) in enumerate(feature_importance_pairs[:10], 1):
    print(f"{rank:2d}. {name}: {importance:.4f}")

정확도(Accuracy): 0.796580093563478
F1 score: 0.6766191195945743
AUC: 0.944196575872389

=== 전체 27개 피처의 변수 중요도 ===
(중요도 순으로 정렬)
 1. 잉여현금흐름 비율: 18.1918
 2. 영업이익률: 12.8760
 3. pcr: 9.6426
 4. 정상영업이익증가율: 5.3027
 5. 차입금의존도: 4.8639
 6. 자산_EBITDA비율: 4.0178
 7. 순이익증가율: 3.4969
 8. psr: 3.3596
 9. 당좌비율: 3.3546
10. x2: 3.1927
11. 재무레버리지효과: 2.5101
12. 매출원가 대 매출액비율: 2.4679
13. 매출액증가율: 2.4667
14. ln(기업업력): 2.4355
15. ln(시가총액): 2.3498
16. x3: 2.3092
17. 총자본순이익률: 2.1947
18. 유동자산_비율: 2.0990
19. 적자_누적_flag: 2.0057
20. ln(매출액): 1.9606
21. 기업유동성위험_flag: 1.8634
22. 유동부채비율: 1.8056
23. 총자본회전률: 1.7539
24. x4: 1.4642
25. 자기자본회전률: 0.8216
26. 유동부채_자본비율: 0.6845
27. 자기자본구성비율: 0.5092

=== 변수 중요도 (상위 10개) ===
 1. 잉여현금흐름 비율: 18.1918
 2. 영업이익률: 12.8760
 3. pcr: 9.6426
 4. 정상영업이익증가율: 5.3027
 5. 차입금의존도: 4.8639
 6. 자산_EBITDA비율: 4.0178
 7. 순이익증가율: 3.4969
 8. psr: 3.3596
 9. 당좌비율: 3.3546
10. x2: 3.1927


## 09. 최종 피쳐명 선택 및 데이터 저장

In [39]:
selected_importance = ["잉여현금흐름 비율","영업이익률","pcr","정상영업이익증가율","차입금의존도","자산_EBITDA비율","순이익증가율","psr","당좌비율","x2"]

cols_to_keep_importance = ['회사명','거래소코드','회계년도'] + selected_importance + ['target_class']
selected_importance_df = df[cols_to_keep_importance]
selected_importance_df.to_excel("data/변수중요도.xlsx", index=False)